In [1]:
import random
from abc import ABC, abstractmethod

class Character(ABC):
    """게임 캐릭터 기본 클래스"""
    
    def __init__(self, name, level=1):
        self.name = name
        self.level = level
        self.max_hp = self.calculate_max_hp()
        self.current_hp = self.max_hp
        self.max_mp = self.calculate_max_mp()
        self.current_mp = self.max_mp
        self.experience = 0
        self.experience_to_next_level = self.calculate_exp_requirement()
    
    @abstractmethod
    def calculate_max_hp(self):
        pass
    
    @abstractmethod
    def calculate_max_mp(self):
        pass
    
    @abstractmethod
    def get_attack_power(self):
        pass
    
    @abstractmethod
    def get_defense(self):
        pass
    
    @abstractmethod
    def special_attack(self, target):
        pass
    
    def calculate_exp_requirement(self):
        return self.level * 100
    
    def attack(self, target):
        damage = max(1, self.get_attack_power() - target.get_defense())
        actual_damage = target.take_damage(damage)
        return f"{self.name}이(가) {target.name}에게 {actual_damage} 피해를 입혔습니다!"
    
    def take_damage(self, damage):
        actual_damage = min(damage, self.current_hp)
        self.current_hp -= actual_damage
        return actual_damage
    
    def heal(self, amount):
        heal_amount = min(amount, self.max_hp - self.current_hp)
        self.current_hp += heal_amount
        return f"{self.name}이(가) {heal_amount} 체력을 회복했습니다!"
    
    def gain_experience(self, exp):
        self.experience += exp
        result = f"{self.name}이(가) {exp} 경험치를 획득했습니다!"
        
        while self.experience >= self.experience_to_next_level:
            self.level_up()
            result += f"\n{self.name}이(가) 레벨업했습니다! (Lv.{self.level})"
        
        return result
    
    def level_up(self):
        self.experience -= self.experience_to_next_level
        self.level += 1
        
        old_max_hp = self.max_hp
        old_max_mp = self.max_mp
        
        self.max_hp = self.calculate_max_hp()
        self.max_mp = self.calculate_max_mp()
        self.experience_to_next_level = self.calculate_exp_requirement()
        
        # 레벨업시 체력/마나 회복
        hp_gain = self.max_hp - old_max_hp
        mp_gain = self.max_mp - old_max_mp
        self.current_hp += hp_gain
        self.current_mp += mp_gain
    
    def is_alive(self):
        return self.current_hp > 0
    
    def get_status(self):
        return (f"{self.name} (Lv.{self.level})\n"
                f"  HP: {self.current_hp}/{self.max_hp}\n"
                f"  MP: {self.current_mp}/{self.max_mp}\n"
                f"  EXP: {self.experience}/{self.experience_to_next_level}")

class Warrior(Character):
    """전사 클래스"""
    
    def calculate_max_hp(self):
        return 100 + (self.level - 1) * 20
    
    def calculate_max_mp(self):
        return 30 + (self.level - 1) * 5
    
    def get_attack_power(self):
        return 25 + (self.level - 1) * 5
    
    def get_defense(self):
        return 15 + (self.level - 1) * 3
    
    def special_attack(self, target):
        if self.current_mp < 10:
            return f"{self.name}의 마나가 부족합니다!"
        
        self.current_mp -= 10
        damage = self.get_attack_power() * 1.5
        actual_damage = target.take_damage(int(damage))
        return f"{self.name}이(가) 강력한 일격으로 {target.name}에게 {actual_damage} 피해를 입혔습니다!"

class Mage(Character):
    """마법사 클래스"""
    
    def calculate_max_hp(self):
        return 60 + (self.level - 1) * 10
    
    def calculate_max_mp(self):
        return 80 + (self.level - 1) * 15
    
    def get_attack_power(self):
        return 35 + (self.level - 1) * 7
    
    def get_defense(self):
        return 8 + (self.level - 1) * 2
    
    def special_attack(self, target):
        if self.current_mp < 20:
            return f"{self.name}의 마나가 부족합니다!"
        
        self.current_mp -= 20
        damage = self.get_attack_power() * 2
        actual_damage = target.take_damage(int(damage))
        return f"{self.name}이(가) 화염구로 {target.name}에게 {actual_damage} 피해를 입혔습니다!"
    
    def heal_spell(self, target):
        if self.current_mp < 15:
            return f"{self.name}의 마나가 부족합니다!"
        
        self.current_mp -= 15
        heal_amount = 30 + self.level * 5
        return target.heal(heal_amount)

class Archer(Character):
    """궁수 클래스"""
    
    def calculate_max_hp(self):
        return 80 + (self.level - 1) * 15
    
    def calculate_max_mp(self):
        return 50 + (self.level - 1) * 8
    
    def get_attack_power(self):
        return 30 + (self.level - 1) * 6
    
    def get_defense(self):
        return 10 + (self.level - 1) * 2
    
    def special_attack(self, target):
        if self.current_mp < 12:
            return f"{self.name}의 마나가 부족합니다!"
        
        self.current_mp -= 12
        # 궁수는 치명타 확률이 높음
        is_critical = random.random() < 0.3
        damage = self.get_attack_power()
        if is_critical:
            damage *= 2.5
            crit_text = " (치명타!)"
        else:
            damage *= 1.2
            crit_text = ""
        
        actual_damage = target.take_damage(int(damage))
        return f"{self.name}이(가) 정밀사격으로 {target.name}에게 {actual_damage} 피해를 입혔습니다!{crit_text}"

# 게임 시뮬레이션
def battle_simulation():
    """전투 시뮬레이션"""
    party = [
        Warrior("김전사"),
        Mage("이마법사"),
        Archer("박궁수")
    ]
    
    print("=== 파티 초기 상태 ===")
    for character in party:
        print(character.get_status())
        print()
    
    # 몇 번의 전투 시뮬레이션
    for battle_round in range(1, 4):
        print(f"=== 전투 {battle_round}라운드 ===")
        
        # 경험치 획득
        exp_gained = random.randint(80, 150)
        for character in party:
            print(character.gain_experience(exp_gained))
        
        print(f"\n전투 {battle_round} 후 상태:")
        for character in party:
            print(character.get_status())
            print()
        
        # 마법사가 파티원 힐링
        mage = party[1]
        for target in party:
            if target.current_hp < target.max_hp * 0.8:
                print(mage.heal_spell(target))
        print()

battle_simulation()


=== 파티 초기 상태 ===
김전사 (Lv.1)
  HP: 100/100
  MP: 30/30
  EXP: 0/100

이마법사 (Lv.1)
  HP: 60/60
  MP: 80/80
  EXP: 0/100

박궁수 (Lv.1)
  HP: 80/80
  MP: 50/50
  EXP: 0/100

=== 전투 1라운드 ===
김전사이(가) 87 경험치를 획득했습니다!
이마법사이(가) 87 경험치를 획득했습니다!
박궁수이(가) 87 경험치를 획득했습니다!

전투 1 후 상태:
김전사 (Lv.1)
  HP: 100/100
  MP: 30/30
  EXP: 87/100

이마법사 (Lv.1)
  HP: 60/60
  MP: 80/80
  EXP: 87/100

박궁수 (Lv.1)
  HP: 80/80
  MP: 50/50
  EXP: 87/100


=== 전투 2라운드 ===
김전사이(가) 132 경험치를 획득했습니다!
김전사이(가) 레벨업했습니다! (Lv.2)
이마법사이(가) 132 경험치를 획득했습니다!
이마법사이(가) 레벨업했습니다! (Lv.2)
박궁수이(가) 132 경험치를 획득했습니다!
박궁수이(가) 레벨업했습니다! (Lv.2)

전투 2 후 상태:
김전사 (Lv.2)
  HP: 120/120
  MP: 35/35
  EXP: 119/200

이마법사 (Lv.2)
  HP: 70/70
  MP: 95/95
  EXP: 119/200

박궁수 (Lv.2)
  HP: 95/95
  MP: 58/58
  EXP: 119/200


=== 전투 3라운드 ===
김전사이(가) 103 경험치를 획득했습니다!
김전사이(가) 레벨업했습니다! (Lv.3)
이마법사이(가) 103 경험치를 획득했습니다!
이마법사이(가) 레벨업했습니다! (Lv.3)
박궁수이(가) 103 경험치를 획득했습니다!
박궁수이(가) 레벨업했습니다! (Lv.3)

전투 3 후 상태:
김전사 (Lv.3)
  HP: 140/140
  MP: 40/40
  EXP: 22/300

이마법사 (Lv.3)
  H

In [ ]:
import random
from abc import ABC, abstractmethod
    # This imports tools for creating abstract base classes.
    # ABC stands for Abstract Base Class — a class that can’t be used directly, only inherited.
    # abstractmethod is a decorator that marks methods as required for subclasses to implement.

# A decorator in Python is like a label or wrapper that adds extra behavior to a function or method — without changing its core logic.

# You use it by placing @something above a function definition.

class Character(ABC):
    """Base class for game characters"""
    def __init__(self, name, level=1):
        self.name=name
        self.level=level
        self.max_hp=self.calculate_max_hp()
        self.current_hp=self.max_hp
        self.max_mp=self.calculate_max_mp()
        self.current_mp=self.max_mp
        self.experience=0
        self.experience_to_next_level=self.calculate_exp_requirement()

    @abstractmethod
    def calculate_max_hp(self):
        pass
    
    @abstractmethod
    def calculate_max_mp(self):
        pass
    
    @abstractmethod
    def get_attack_power(self):
        pass
    
    @abstractmethod
    def get_defense(self):
        pass
    
    @abstractmethod
    def special_attack(self, target):
        pass
# The pass statement in Python is like saying: “I’m putting a placeholder here — I’ll fill it in later.

    